# Data Pre Processing

This is a notebook for intaking a raw data file, preprocessing it, and exporting it in a format in which it can easily be labeled.

In [18]:
import pandas as pd

In [19]:
df = pd.read_csv("data/directoryident.tsv", sep="\t")
df.head()

,dn,lastname,givenname,middle,dob,address
0,"uid=madanafl,ou=People,dc=umich,dc=edu",Madanay,Farrah,L,1990091200Z,NaN
1,"uid=jocbran,ou=People,dc=umich,dc=edu",Branham,Jocelyn,NaN,20050616000000Z,{addr1=345A Infantry Ter}:{city=San Francisco}...
2,"uid=stoicabr,ou=People,dc=umich,dc=edu",Stoica,Brooke,NaN,20031007000000Z,{addr1=9051 Salem Farms Drive}:{city=South Lyo...
3,"uid=heavens,ou=People,dc=umich,dc=edu",Smith,Heaven,NaN,19880104000000Z,{addr1=6655 Jackson Rd Unit 191}:{city=Ann Arb...
4,"uid=funnim,ou=People,dc=umich,dc=edu",Funni,Matthew,NaN,1989121900Z,{addr1=2095 pauls way}:{city=Commerce twp}:{st...


## Preprocessing

### Cleanup

In [20]:
# Add unique ID column
# Uses the row number of original csv file as unique ID
df['Unique ID'] = df.index + 1

In [21]:
# Turn all NaN values into empty strings
df['address'] = df['address'].fillna(" ")
df['middle'] = df['middle'].fillna(" ")

In [22]:
df.head()

,dn,lastname,givenname,middle,dob,address,Unique ID
0,"uid=madanafl,ou=People,dc=umich,dc=edu",Madanay,Farrah,L,1990091200Z,,1
1,"uid=jocbran,ou=People,dc=umich,dc=edu",Branham,Jocelyn,,20050616000000Z,{addr1=345A Infantry Ter}:{city=San Francisco}...,2
2,"uid=stoicabr,ou=People,dc=umich,dc=edu",Stoica,Brooke,,20031007000000Z,{addr1=9051 Salem Farms Drive}:{city=South Lyo...,3
3,"uid=heavens,ou=People,dc=umich,dc=edu",Smith,Heaven,,19880104000000Z,{addr1=6655 Jackson Rd Unit 191}:{city=Ann Arb...,4
4,"uid=funnim,ou=People,dc=umich,dc=edu",Funni,Matthew,,1989121900Z,{addr1=2095 pauls way}:{city=Commerce twp}:{st...,5


In [23]:
# Convert all columns to string
df = df.astype(str)

In [24]:
# Function to parse a singular dn field
# Returns the uid in the dn
def parse_dn(dn):
    dn = dn.split(',')
    dn = [x.split('=') for x in dn]
    dn = {x[0].strip(): x[1].strip() for x in dn}
    return dn["uid"]

In [25]:
# Function to concatenate every value in the address 
# We want to concatenate all values inside curly braces
def concat_address(address):
    if address != ' ':
        address = address.split('}:{')
        try: 
            address = [x.split('=')[1] for x in address]
        except:
            return "DELETE"
        address = ' '.join(address)
    return address

In [26]:
# Use vectorization to set new values for dn and address columns
df['dn'] = df['dn'].apply(parse_dn)
df['address'] = df['address'].apply(concat_address)
df['dob'] = df['dob'].apply(lambda x : x[:8])

In [27]:
if "DELETE" in df['address'].values:
    df = df[df['address'] != "DELETE"]

df.head()

,dn,lastname,givenname,middle,dob,address,Unique ID
0,madanafl,Madanay,Farrah,L,19900912,,1
1,jocbran,Branham,Jocelyn,,20050616,345A Infantry Ter San Francisco CA 94129-1280 ...,2
2,stoicabr,Stoica,Brooke,,20031007,9051 Salem Farms Drive South Lyon MI 48178 UNI...,3
3,heavens,Smith,Heaven,,19880104,6655 Jackson Rd Unit 191 Ann Arbor MI 48103-95...,4
4,funnim,Funni,Matthew,,19891219,2095 pauls way Commerce twp MI 48390 UNITED ST...,5


In [28]:
# Change column names
df.columns = ['ID', 'Last Name', 'First Name', 'Middle Name', 'Date of Birth', 'Address', 'Unique ID']
# Reorder columns
df = df[['Unique ID', 'ID', 'First Name', 'Middle Name', 'Last Name', 'Date of Birth', 'Address']]

In [29]:
df.head()

,Unique ID,ID,First Name,Middle Name,Last Name,Date of Birth,Address
0,1,madanafl,Farrah,L,Madanay,19900912,
1,2,jocbran,Jocelyn,,Branham,20050616,345A Infantry Ter San Francisco CA 94129-1280 ...
2,3,stoicabr,Brooke,,Stoica,20031007,9051 Salem Farms Drive South Lyon MI 48178 UNI...
3,4,heavens,Heaven,,Smith,19880104,6655 Jackson Rd Unit 191 Ann Arbor MI 48103-95...
4,5,funnim,Matthew,,Funni,19891219,2095 pauls way Commerce twp MI 48390 UNITED ST...


### Similarity Measures

In [30]:
# Sample 200 records from df into new df
df_sample = df.sample(100)

In [31]:
df_sample.head()

,Unique ID,ID,First Name,Middle Name,Last Name,Date of Birth,Address
532718,532719,langla,Lisa,Ann,Lang,19630109,
608285,608286,robinal,Robin,Adelson,Little,19521130,4635 Mulberry Woods Circle 48105 USA Ann Arbor...
377109,377110,omasson,Ondrea,Rae,Masson,19790609,28959 Bock St Garden City MI 48135-2806 UNITED...
346971,346972,schu,Christopher,Alan,Schumaker,19640802,951 N Riverside St Clair MI 48079 USA}
638319,638320,rdamphou,Renee,Rochelle,Robertson,19750118,9545 Mcgregor Road Pinckney MI 48169 UNITED ST...


In [32]:
# Remove sampled records from df
df = df.drop(df_sample.index)

With a sample df, we can now build a similarity measure df by comparing each record in the sample df with the other records.

In [33]:
import Levenshtein as lev

In [34]:
"""
Returns normalized levenshtein distance between two strings
"""
def levenshtein_distance(
        s1: str,
        s2: str
) -> int:
    distance = lev.distance(s1, s2)
    try:
        return 1 - distance / float(max(len(s1), len(s2)))
    except ZeroDivisionError:
        return 0

In [35]:
"""
Returns Levenshtein distance between each field of two rows
Assumes following format for rows:
    row = pd.Series([First Name, Middle Name, Last Name, DOB, Address, ID])
"""
def row_similarity(
        row_1: pd.Series, 
        df_2: pd.DataFrame
    ) -> pd.DataFrame:

    # Compute similarity measures for each column using Levenshtein distance
    first_name_similarity = df_2['First Name'].apply(lambda x : levenshtein_distance(row_1['First Name'], x))
    middle_name_similarity = df_2['Middle Name'].apply(lambda x : levenshtein_distance(row_1['Middle Name'], x))
    last_name_similarity = df_2['Last Name'].apply(lambda x : levenshtein_distance(row_1['Last Name'], x))
    dob_similarity = df_2['Date of Birth'].apply(lambda x : levenshtein_distance(row_1['Date of Birth'], x))
    address_similarity = df_2['Address'].apply(lambda x : levenshtein_distance(row_1['Address'], x))
    id_similarity = df_2['ID'].apply(lambda x : levenshtein_distance(row_1['ID'], x))

    # Return DataFrame with similarity measures
    return pd.DataFrame({
        'UNIQ_ID1': row_1['Unique ID'],
        'UNIQ_ID2': df_2['Unique ID'],
        'ID1': row_1['ID'],
        'ID2': df_2['ID'],
        'ID Similarity': id_similarity,
        'First Name 1' : row_1['First Name'],
        'First Name 2' : df_2['First Name'],
        'First Name Similarity': first_name_similarity,
        'Middle Name 1' : row_1['Middle Name'],
        'Middle Name 2' : df_2['Middle Name'],
        'Middle Name Similarity': middle_name_similarity,
        'Last Name 1' : row_1['Last Name'],
        'Last Name 2' : df_2['Last Name'],
        'Last Name Similarity': last_name_similarity,
        'Date of Birth 1' : row_1['Date of Birth'],
        'Date of Birth 2' : df_2['Date of Birth'],
        'Date of Birth Similarity': dob_similarity,
        'Address 1' : row_1['Address'],
        'Address 2' : df_2['Address'],
        'Address Similarity': address_similarity
    })


In [36]:
"""
Builds similarity measure between records in two df
Creates a new df from the two df with the following columns:
    - ID1: ID of record in df1
    - ID2: ID of record in df2
    - First Name Similarity: Normalized levenshtein distance between first names
    - Middle Name Similarity: Normalized levenshtein distance between middle names
    - Last Name Similarity: Normalized levenshtein distance between last names
    - Date of Birth Similarity: Normalized levenshtein distance between dates of birth
    - Address Similarity: Normalized levenshtein distance between addresses
    - ID Similarity: Normalized levenshtein distance between IDs
"""
def build_similarity_df (
        df_1: pd.DataFrame,
        df_2: pd.DataFrame
) -> pd.DataFrame:
    new_df = pd.DataFrame(columns=[ 'UNIQ_ID1',
                                    'UNIQ_ID2',
                                    'ID1', 
                                   'ID2', 
                                   'ID Similarity'
                                   'First Name 1',
                                   'First Name 2',
                                   'First Name Similarity', 
                                   'Middle Name 1',
                                    'Middle Name 2',
                                   'Middle Name Similarity',
                                    'Last Name 1',
                                    'Last Name 2', 
                                   'Last Name Similarity', 
                                    'Date of Birth 1',
                                    'Date of Birth 2',
                                   'Date of Birth Similarity', 
                                    'Address 1',
                                    'Address 2',
                                   'Address Similarity'])

    # Convert ID columns to string
    df_1["ID"] = df_1["ID"].astype(str)
    df_2["ID"] = df_2["ID"].astype(str)

    def apply_row_similarity(row, new_df):
        sim = row_similarity(row, df_2)
        new_df = pd.concat([new_df, sim], ignore_index=True)
        return new_df

    # Use vectorization to compute similarity between each row in df_1 and df_2
    new_df = df_1.apply(apply_row_similarity, args=(new_df,), axis=1).reset_index(drop=True)

    # New df is a series of dfs, so we need to concatenate them
    new_df = pd.concat(new_df.to_list(), ignore_index=True)

    return new_df

In [37]:
similarity_df = build_similarity_df(df_sample,  df)
similarity_df.head()

,UNIQ_ID1,UNIQ_ID2,ID1,ID2,ID SimilarityFirst Name 1,First Name 2,First Name Similarity,Middle Name 1,Middle Name 2,Middle Name Similarity,...,Last Name 2,Last Name Similarity,Date of Birth 1,Date of Birth 2,Date of Birth Similarity,Address 1,Address 2,Address Similarity,ID Similarity,First Name 1
0,532719,1,langla,madanafl,NaN,Farrah,0.166667,Ann,L,0.0,...,Madanay,0.285714,19630109,19900912,0.375,,,1.000000,0.250000,Lisa
1,532719,2,langla,jocbran,NaN,Jocelyn,0.000000,Ann,,0.0,...,Branham,0.285714,19630109,20050616,0.125,,345A Infantry Ter San Francisco CA 94129-1280 ...,0.015625,0.142857,Lisa
2,532719,3,langla,stoicabr,NaN,Brooke,0.000000,Ann,,0.0,...,Stoica,0.000000,19630109,20031007,0.250,,9051 Salem Farms Drive South Lyon MI 48178 UNI...,0.016393,0.125000,Lisa
3,532719,4,langla,heavens,NaN,Heaven,0.000000,Ann,,0.0,...,Smith,0.000000,19630109,19880104,0.625,,6655 Jackson Rd Unit 191 Ann Arbor MI 48103-95...,0.014925,0.142857,Lisa
4,532719,5,langla,funnim,NaN,Matthew,0.000000,Ann,,0.0,...,Funni,0.200000,19630109,19891219,0.375,,2095 pauls way Commerce twp MI 48390 UNITED ST...,0.018182,0.166667,Lisa


### Discretization

In [ ]:
# Export similarity_df to csv
similarity_df.to_csv('data/unlabeledident.csv', index=False)